# Dressmaker - Samples

In [ ]:
library(tidyverse)
library(DBI)
library(getPass)
drv <- switch(Sys.info()['sysname'],
             Windows="PostgreSQL Unicode(x64)",
             Darwin="/usr/local/lib/psqlodbcw.so",
             Linux="PostgreSQL")
con <- dbConnect(
  odbc::odbc(),
  driver = drv,
  Server = "localhost",
  Database = "sqlzoo",
  UID = "postgres",
  PWD = getPass("Password?"),
  Port = 5432
)
options(repr.matrix.max.rows=20)

## 1.
The "central" table in this database is order_line - every garment ordered takes one line in this table. Many of the fields in this table are references to other tables. The fields of this table have the following meaning: order_ref

This is a link to the dress_order table. We can join the dress_order table to find information such as the the date of the order and the customer number for a given garment order.

- line_no

The line number is used to distinguish different items on the same order - for example order number 5 has three lines - 1, 2 and 3.

- ol_style

Indicates the article of clothing ordered. For example ol_style 1 indicates trousers - we can see this by joining to the garments table. Line 1 in the garment table is trousers.

- ol_size

The size of the item ordered is given here - this is particularly important when it comes to working out how much material is required to build the item. We can see from the quantities table that trousers (style 1) in size 8 takes 2.7 meters - whereas trousers in size 12 needs 2.8 meters.

- ol_material

Each order specifies the material to be used. We can join to the material table to find a description or cost per meter. Material 1 is Silk, Black, Plain costing £7 per meter.

```sql
show create table order_line
```

## 2.
A sample join:

In order to translate the numbers in order_line into meaningful values we need to join a related table. For example if we want to access the descriptions of the materials we need to join the material table.

To achieve the join we include the table material on the FROM line and the join condition as a WHERE clause.

For each pair of tables there is a join condition between them (if they are linked). To find the join condition between order_line and material we look at the order_line table CREATE statement and notice the line that specifies that ol_material references the material table. This link will always be to the primary key of material table.

```sql
CREATE TABLE order_line (
  order_ref	INTEGER	NOT NULL REFERENCES dress_order
 ,line_no	INTEGER	NOT NULL
 ,ol_style	INTEGER	REFERENCES garment
 ,ol_size	INTEGER	NOT NULL
 ,ol_material	INTEGER	REFERENCES material
 ,PRIMARY KEY (order_ref, line_no)
 ,FOREIGN KEY (ol_style, ol_size) REFERENCES quantities
 );
SELECT order_ref, line_no, fabric, colour, pattern, cost
  FROM order_line, material
 WHERE ol_material = material_no
```

## 3.
To get a description of the garment we need to join the garment table. The join condition is that the ol_style in order_line matches the style_no in garment.

```sql
SELECT order_ref, line_no, description
  FROM order_line, garment
 WHERE ol_style = style_no
```

## 4.
If we need both the description and the fabric we can join both material and garment to the order_line table. The join conditions are combined with "AND"

```sql
SELECT order_ref, line_no, fabric, description
  FROM order_line, material, garment
 WHERE ol_style = style_no
   AND ol_material = material_no
```

## 5.
The quantities table tells us how much material is required for every garment for every size available. The join between the order_line and quantities is unusual in that it involves two fields. This can be seen by the fact that quantities has a composite key.

```sql
SELECT order_ref, line_no, quantity
  FROM order_line, quantities
 WHERE ol_style = style_q
   AND ol_size  = size_q
```

## 6.
Customers place orders - each order contains many lines - each line of the order refers to a garment:

```sql
SELECT c_name, order_date, order_no, line_no
   FROM jmcust, dress_order, order_line
  WHERE          jmcust.c_no = dress_order.cust_no
  AND   dress_order.order_no = order_line.order_ref
```

## 7.
There's also a dress maker table, and a table called construction which gives you information about who made which order and when:

```sql
SELECT d_no, d_name, construction.order_ref, construction.line_ref, start_date, finish_date
 FROM dressmaker, order_line, construction
 WHERE  d_no=maker  
 AND order_line.order_ref=construction.order_ref 
 AND order_line.line_no=construction.line_ref
```